In [2]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import get_scheduler

##### Own
from train_utils import make_optimizer
from vision_transformer import VisionTransformer
from train_utils import cifar_utils

In [2]:
# batch_size = 4
# cifar = load_dataset("uoft-cs/cifar100")
# train_label_type = "coarse" # cfg["dataset"]["train_label_type"]

# train_dataloader, test_dataloader = cifar_utils.dataloader_from_dataset_dict(cifar, batch_size=batch_size)
# label2id_coarse, id2label_coarse = cifar_utils.get_cifar_label_dicts(cifar, train_label_type)

# model = VisionTransformer(
#         image_size=32, use_linear_patch=True, num_classes=len(label2id_coarse.keys()))

# num_epochs = 3 # TODO: set a param in the config file
# lr = 0.003
# num_training_steps = num_epochs * len(train_dataloader)

# optimizer = make_optimizer(optimizer_name='adamw',model=model, lr=0.003)
# lr_scheduler = get_scheduler(
# name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
# )
# loss_function = nn.CrossEntropyLoss()

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model = model.to(device)

In [3]:
batch_size = 2
cifar_small = load_dataset("uoft-cs/cifar100", split="train[:10]")
train_label_type = "coarse" # cfg["dataset"]["train_label_type"]

# train_dataloader, test_dataloader = cifar_utils.dataloader_from_dataset_dict(cifar, batch_size=batch_size)
train_dataloader= cifar_utils.dataloader_from_dataset(cifar_small, batch_size=batch_size)
label2id_coarse, id2label_coarse = cifar_utils.get_cifar_label_dicts(cifar_small, train_label_type)

model = VisionTransformer(
        image_size=32, use_linear_patch=True, num_classes=len(label2id_coarse.keys()))

num_epochs = 1 # TODO: set a param in the config file
lr = 0.003
num_training_steps = num_epochs * len(train_dataloader)

optimizer = make_optimizer(optimizer_name='adamw',model=model, lr=0.003)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
    )
loss_function = nn.CrossEntropyLoss()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

In [3]:
# single epoch training loop
model.train()
epoch_train_loss = 0

for batch_idx, batch in enumerate(train_dataloader):
    # transfer batch to device
    batch = {k: v.to(device) for k, v in batch.items()}

    # forward pass and loss calculation
    outputs = model(batch["pixel_values"])
    loss = loss_function(outputs, batch["coarse_label"])
    loss.backward()
    epoch_train_loss += loss.item()

    # backward pass
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
epoch_train_loss/=len(train_dataloader)

# logger.info(f'Validating at epoch {epoch}')
epoch_val_loss = 0
with torch.no_grad():
    model.eval()
    total_examples, correct_predictions= 0.0, 0.0
    for batch_idx, batch in enumerate(train_dataloader):
        # transfer batch to device
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(batch["pixel_values"])
        
        epoch_val_loss += loss_function(outputs, batch["coarse_label"]).item() # retrieve only the scalar value
        pred_labels = outputs.argmax(dim=1)
        
        total_examples += float(len(batch['coarse_label']))
        correct_predictions += float((batch["coarse_label"] == pred_labels).sum().item())

    acc = correct_predictions / total_examples
    epoch_val_loss /= len(train_dataloader)

    print(f'-- train loss {train_loss:.3f} -- validation accuracy {acc:.3f} -- validation loss: {epoch_val_loss:.3f}')
    if epoch_val_loss <= best_val_loss and save_model:
        torch.save(model.state_dict(), 'model.pth')
        best_val_loss = epoch_val_loss

In [25]:
epoch_val_loss = 0
correct_predictions = 0
# model = model.train()

In [53]:
print(len(train_dataloader))

5


In [52]:
correct_predictions = 0
batch = next(iter(train_dataloader))
batch = {k: v.to(device) for k, v in batch.items()}
print(f"batch_size: {batch['pixel_values'].shape}")
print(f"batch labels: {batch['coarse_label']}")

outputs = model(batch["pixel_values"])

loss = loss_function(outputs, batch["coarse_label"])
epoch_val_loss += loss.item() # retrieve only the scalar value
pred_labels = outputs.argmax(dim=1)

print(f"outputs: {outputs.shape}")
print(f"loss: {loss}")
print(f"pred_labels: {pred_labels}")

correct_predictions += int((batch["coarse_label"] == pred_labels).sum().item())
print(f"correct: {correct_predictions}")

batch_size: torch.Size([2, 3, 32, 32])
batch labels: tensor([11,  4], device='cuda:0')
outputs: torch.Size([2, 20])
loss: 2.995732307434082
pred_labels: tensor([ 1., 11.], device='cuda:0')
correct: 0


In [ ]:

for batch_idx, batch in enumerate(train_dataloader):
    print(batch["coarse_label"])
    if batch_idx > 2:
        break
# batch = next(iter(train_dataloader))
# # print((batch))
# outputs = model(batch["pixel_values"])

In [18]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

In [ ]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")